# An overview of the discrepancies

## 1.Exploration

In [ ]:
#!pip install snowflake-connector-python
#!pip install snowflake-connector-python[pandas]
import snowflake.connector
from snowflake.connector import cursor

In [ ]:
ctx = snowflake.connector.connect(
      user = '',
      password = '',
      account = ''
 )

cursor = ctx.cursor()

- Preview data

In [ ]:
# Show the result of null percentage
def combine_res_des(res, des):
    result_dict = dict()
    for i in range(len(res[0])):
        result_dict[list(des[i])[0]] = res[0][i]
    return result_dict

In [ ]:
sql_MLS_NULL = """"""
cursor.execute(sql_MLS_NULL)
res = cursor.fetchall()
des = cursor.description

In [ ]:
#Show the null percentage of MLS
combine_res_des(res, des)

In [ ]:
sql_datatree_NULL = """"""
cursor.execute(sql_datatree_NULL)
res = cursor.fetchall()
des = cursor.description

In [ ]:
#Show the null percentage of DATATREE
combine_res_des(res, des)

## 2.Address Completely match

In [ ]:
import pandas as pd
sql_join = """"""
df_nyc = pd.read_sql(sql_join, con=ctx)

In [ ]:
df_nyc.columns = ['MLS_ADDRESS', 'MLS_NAME', 'CITY', 'DATATREE_ADDRESS', 'DATATREE_NAME1', 'DATATREE_NAME2']

- NULL values in the MLS_NAME

In [ ]:
df_nyc

In [ ]:
df_nyc = df_nyc[df_nyc['MLS_NAME'].notnull()]

- 'withheld' in MLS_NAME

In [ ]:
df_nyc

- There are addresses with different names 

In [ ]:
df_nyc.DATATREE_ADDRESS.value_counts()

In [ ]:
df_nyc[df_nyc['MLS_ADDRESS'] == '635 w 42nd st new york'] 

- Compute the fuzzy score of names

In [ ]:
#!pip install thefuzz
from thefuzz import fuzz

In [ ]:
fuzz.ratio('withheld', 'cassisi anastasia')

In [ ]:
df_nyc['name1_fuzz_ratio'] = df_nyc.apply(lambda row: fuzz.ratio(row['MLS_NAME'], row['DATATREE_NAME1']), axis=1)

In [ ]:
df_nyc['name2_fuzz_ratio'] = df_nyc.apply(lambda row: fuzz.ratio(row['MLS_NAME'], row['DATATREE_NAME2']), axis=1)

In [ ]:
(df_nyc['name1_fuzz_ratio'] == 100).sum()

In [ ]:
df_nyc[(df_nyc['name1_fuzz_ratio'] < 100)&(df_nyc['name1_fuzz_ratio'] > 80)]
#punctuation, abbreviation, space, part of the another name, misspelling

In [ ]:
df_nyc[(df_nyc['name1_fuzz_ratio'] < 80)&(df_nyc['name1_fuzz_ratio'] > 50)]
#only last name/first name in one table; reverse order;

In [ ]:
df_nyc[df_nyc['name1_fuzz_ratio'] < 50]
#completely mismatching